# Pre-Proceso (1)


Validación de los datos
* Revisión de valores anómalos
* Revisión de valores faltantes
* Inconsistencias


Definir SparkContext y SQLContext

In [1]:
#from pyspark import SparkContext
#sc = SparkContext()
#from pyspark.sql import SQLContext
#sqlContext=SQLContext(sc)


In [2]:
bd = sqlContext.read.format("com.databricks.spark.csv"
).option("header", "true"
).load("On_Time_On_Time_Performance_2016_12.csv", 
       inferSchema=True)

In [3]:
bd=bd.select('YEAR','MONTH','DAY_OF_MONTH','DAY_OF_WEEK','CRS_DEP_TIME',
             'OP_UNIQUE_CARRIER', 'TAIL_NUM', 'ARR_DELAY','DEP_DELAY',
             'ORIGIN','DEST','DISTANCE','CANCELLED', 'DIVERTED','CARRIER_DELAY',
             'WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY',
             'LATE_AIRCRAFT_DELAY') 
# bd=bd.select('Year','Month','DayofMonth','DayOfWeek','CRSDepTime',
#              'UniqueCarrier', 'TailNum', 'ArrDelay','DepDelay','Origin','Dest',
#              'Distance','Cancelled', 'Diverted','CarrierDelay','WeatherDelay',
#              'NASDelay','SecurityDelay','LateAircraftDelay') 

In [6]:
bd2 = bd.filter( bd.ORIGIN.isin('ATL','ORD','DEN','LAX','DFW','SFO','PHX','LAS') & bd.DEST.isin(
    'ATL','ORD','DEN','LAX','DFW','SFO','PHX','LAS') )

sqlContext.registerDataFrameAsTable(bd2, "bd2")

## Validación

### Valores Anómalos

In [8]:
bd2.describe('ARR_DELAY').show()

+-------+------------------+
|summary|         ARR_DELAY|
+-------+------------------+
|  count|             30466|
|   mean|13.300695857677411|
| stddev|53.509467381637556|
|    min|             -53.0|
|    max|            2028.0|
+-------+------------------+



In [9]:
bd2.describe().show()

+-------+------+-----+------------------+-----------------+------------------+------------------+-----------------+-----------------+--------------------+--------------------+-----------------+------------------+------------------+--------------------+-------------------+
|summary|  YEAR|MONTH|      DAY_OF_MONTH|      DAY_OF_WEEK|      CRS_DEP_TIME|         ARR_DELAY|        DEP_DELAY|         DISTANCE|           CANCELLED|            DIVERTED|    CARRIER_DELAY|     WEATHER_DELAY|         NAS_DELAY|      SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+-------+------+-----+------------------+-----------------+------------------+------------------+-----------------+-----------------+--------------------+--------------------+-----------------+------------------+------------------+--------------------+-------------------+
|  count| 30965|30965|             30965|            30965|             30965|             30466|            30501|            30965|               30965|               30965|      

Exportamos la tabla a Pandas para mejorar la visualización

In [10]:
bd2.describe().toPandas().set_index('summary').transpose()

summary,count,mean,stddev,min,max
YEAR,30965,2016.0,0.0,2016,2016
MONTH,30965,12.0,0.0,12,12
DAY_OF_MONTH,30965,15.943484579363798,8.876012086360825,1,31
DAY_OF_WEEK,30965,4.008719522041014,1.925961691515872,1,7
CRS_DEP_TIME,30965,1336.3809785241401,507.696934202855,5,2359
ARR_DELAY,30466,13.300695857677411,53.509467381637556,-53.0,2028.0
DEP_DELAY,30501,17.28562997934494,49.93901250742802,-21.0,2040.0
DISTANCE,30965,932.5556596156952,547.1294895897906,236.0,2139.0
CANCELLED,30965,0.015533667043436138,0.1236643283549586,0.0,1.0
DIVERTED,30965,5.813014694009365E-4,0.024103574850036374,0.0,1.0


### Inconsistencias

Variable Distance en función de los diferentes aeropuertos 

In [11]:
bd2.select('ORIGIN','DEST','DISTANCE').groupBy('ORIGIN','DEST').mean().show()

+------+----+-------------+
|ORIGIN|DEST|avg(DISTANCE)|
+------+----+-------------+
|   ATL| ORD|        606.0|
|   LAS| LAX|        236.0|
|   ATL| LAS|       1747.0|
|   LAS| ATL|       1747.0|
|   ATL| LAX|       1946.0|
|   DEN| PHX|        602.0|
|   ORD| LAS|       1514.0|
|   LAX| DEN|        862.0|
|   DEN| DFW|        641.0|
|   ORD| LAX|       1744.0|
|   DFW| ORD|        802.0|
|   DEN| SFO|        967.0|
|   ORD| ATL|        606.0|
|   DFW| LAS|       1055.0|
|   LAS| DEN|        628.0|
|   DFW| LAX|       1235.0|
|   PHX| ORD|       1440.0|
|   DFW| ATL|        731.0|
|   ATL| DEN|       1199.0|
|   PHX| LAS|        255.0|
+------+----+-------------+
only showing top 20 rows



In [12]:
sqlContext.sql("select Origin, Dest, count(*) as n, mean(Distance) as mean, \
                min(Distance) as min , max(Distance) as max,  \
                max(Distance) - min(Distance) as dif, \
                stddev(Distance) as stdev \
                from bd2 \
                group by Origin, Dest" ).show()

+------+----+---+------+------+------+---+--------------------+
|Origin|Dest|  n|  mean|   min|   max|dif|               stdev|
+------+----+---+------+------+------+---+--------------------+
|   ATL| ORD|530| 606.0| 606.0| 606.0|0.0|                 0.0|
|   LAS| LAX|998| 236.0| 236.0| 236.0|0.0|                 0.0|
|   ATL| LAS|368|1747.0|1747.0|1747.0|0.0|                 0.0|
|   LAS| ATL|366|1747.0|1747.0|1747.0|0.0|                 0.0|
|   ATL| LAX|541|1946.0|1946.0|1946.0|0.0|9.511118584115369...|
|   DEN| PHX|680| 602.0| 602.0| 602.0|0.0|                 0.0|
|   ORD| LAS|299|1514.0|1514.0|1514.0|0.0|                 0.0|
|   LAX| DEN|578| 862.0| 862.0| 862.0|0.0|                 0.0|
|   DEN| DFW|453| 641.0| 641.0| 641.0|0.0|                 0.0|
|   ORD| LAX|742|1744.0|1744.0|1744.0|0.0|                 0.0|
|   DFW| ORD|578| 802.0| 802.0| 802.0|0.0|                 0.0|
|   DEN| SFO|674| 967.0| 967.0| 967.0|0.0|                 0.0|
|   ORD| ATL|541| 606.0| 606.0| 606.0|0.

In [14]:
sqlContext.sql("select Origin, Dest, count(*) as n, mean(Distance) as mean, \
                min(Distance) as min , max(Distance) as max,  \
                max(Distance) - min(Distance) as dif, \
                stddev(Distance) as stdev \
                from bd2 \
                group by Origin, Dest" ).toPandas()

,Origin,Dest,n,mean,min,max,dif,stdev
0,ATL,ORD,530,606.0,606.0,606.0,0.0,0.000000e+00
1,LAS,LAX,998,236.0,236.0,236.0,0.0,0.000000e+00
2,ATL,LAS,368,1747.0,1747.0,1747.0,0.0,0.000000e+00
3,LAS,ATL,366,1747.0,1747.0,1747.0,0.0,0.000000e+00
4,ATL,LAX,541,1946.0,1946.0,1946.0,0.0,9.511119e-14
5,DEN,PHX,680,602.0,602.0,602.0,0.0,0.000000e+00
6,ORD,LAS,299,1514.0,1514.0,1514.0,0.0,0.000000e+00
7,LAX,DEN,578,862.0,862.0,862.0,0.0,0.000000e+00
8,DEN,DFW,453,641.0,641.0,641.0,0.0,0.000000e+00
9,ORD,LAX,742,1744.0,1744.0,1744.0,0.0,0.000000e+00


### Valores Faltantes

In [15]:
bd2.describe().toPandas().transpose()[0]

summary                count
YEAR                   30965
MONTH                  30965
DAY_OF_MONTH           30965
DAY_OF_WEEK            30965
CRS_DEP_TIME           30965
ARR_DELAY              30466
DEP_DELAY              30501
DISTANCE               30965
CANCELLED              30965
DIVERTED               30965
CARRIER_DELAY           8401
WEATHER_DELAY           8401
NAS_DELAY               8401
SECURITY_DELAY          8401
LATE_AIRCRAFT_DELAY     8401
Name: 0, dtype: object

Valores faltantes en ArrDelay

In [17]:
bd2.select('ARR_DELAY').count()

30965

In [18]:
from pyspark.sql.functions import col
bd2.describe('YEAR','ARR_DELAY').filter(col("summary") == "count").collect()

[Row(summary='count', YEAR='30965', ARR_DELAY='30466')]

In [19]:
bd2.select('ARR_DELAY').count() - bd2.select('ARR_DELAY').dropna().count()

499

Los valores faltantes en ArrDelay son causados por vuelos cancelados o desviados

In [20]:
bd2.select('CANCELLED','DIVERTED').groupBy().sum().show()

+--------------+-------------+
|sum(CANCELLED)|sum(DIVERTED)|
+--------------+-------------+
|         481.0|         18.0|
+--------------+-------------+



In [21]:
bd3 = bd2.filter(bd2.CANCELLED==0
).filter(bd2.DIVERTED==0)

In [22]:
bd3.describe().toPandas().transpose()[0]

summary                count
YEAR                   30466
MONTH                  30466
DAY_OF_MONTH           30466
DAY_OF_WEEK            30466
CRS_DEP_TIME           30466
ARR_DELAY              30466
DEP_DELAY              30466
DISTANCE               30466
CANCELLED              30466
DIVERTED               30466
CARRIER_DELAY           8401
WEATHER_DELAY           8401
NAS_DELAY               8401
SECURITY_DELAY          8401
LATE_AIRCRAFT_DELAY     8401
Name: 0, dtype: object

Valores faltantes en Tipos de *Delay*

In [24]:
bd3.filter(bd3.ARR_DELAY<0).select('ARR_DELAY','CARRIER_DELAY','WEATHER_DELAY',
                                  'NAS_DELAY','SECURITY_DELAY',
                                  'LATE_AIRCRAFT_DELAY').show(20)

+---------+-------------+-------------+---------+--------------+-------------------+
|ARR_DELAY|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+---------+-------------+-------------+---------+--------------+-------------------+
|    -19.0|         null|         null|     null|          null|               null|
|    -12.0|         null|         null|     null|          null|               null|
|    -14.0|         null|         null|     null|          null|               null|
|    -15.0|         null|         null|     null|          null|               null|
|    -23.0|         null|         null|     null|          null|               null|
|    -16.0|         null|         null|     null|          null|               null|
|     -3.0|         null|         null|     null|          null|               null|
|    -16.0|         null|         null|     null|          null|               null|
|    -11.0|         null|         null|     null|          null| 

In [25]:
bd3.filter(bd3.ARR_DELAY>=0).select('ARR_DELAY','CARRIER_DELAY','WEATHER_DELAY',
                                  'NAS_DELAY','SECURITY_DELAY',
                                  'LATE_AIRCRAFT_DELAY').show(20)

+---------+-------------+-------------+---------+--------------+-------------------+
|ARR_DELAY|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+---------+-------------+-------------+---------+--------------+-------------------+
|      7.0|         null|         null|     null|          null|               null|
|      0.0|         null|         null|     null|          null|               null|
|      3.0|         null|         null|     null|          null|               null|
|      0.0|         null|         null|     null|          null|               null|
|      3.0|         null|         null|     null|          null|               null|
|     18.0|          7.0|          0.0|     11.0|           0.0|                0.0|
|      8.0|         null|         null|     null|          null|               null|
|    119.0|        119.0|          0.0|      0.0|           0.0|                0.0|
|     15.0|         15.0|          0.0|      0.0|           0.0| 

In [26]:
bd3.filter(bd3.ARR_DELAY==15).select('ARR_DELAY','CARRIER_DELAY','WEATHER_DELAY',
                                  'NAS_DELAY','SECURITY_DELAY',
                                  'LATE_AIRCRAFT_DELAY').show(20)

+---------+-------------+-------------+---------+--------------+-------------------+
|ARR_DELAY|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+---------+-------------+-------------+---------+--------------+-------------------+
|     15.0|         15.0|          0.0|      0.0|           0.0|                0.0|
|     15.0|          0.0|          0.0|      0.0|           0.0|               15.0|
|     15.0|          0.0|          0.0|     15.0|           0.0|                0.0|
|     15.0|          0.0|          0.0|     15.0|           0.0|                0.0|
|     15.0|          9.0|          0.0|      3.0|           0.0|                3.0|
|     15.0|          1.0|          0.0|      0.0|           0.0|               14.0|
|     15.0|          0.0|          0.0|     15.0|           0.0|                0.0|
|     15.0|          7.0|          0.0|      8.0|           0.0|                0.0|
|     15.0|         15.0|          0.0|      0.0|           0.0| 

In [28]:
bd3.filter(bd3.ARR_DELAY>=15).select('ARR_DELAY','CARRIER_DELAY','WEATHER_DELAY',
                                  'NAS_DELAY','SECURITY_DELAY',
                                  'LATE_AIRCRAFT_DELAY').count()

8401

In [29]:
bd4 = bd3.na.fill({'CARRIER_DELAY':0, 'WEATHER_DELAY':0,'NAS_DELAY':0,
                   'SECURITY_DELAY':0, 'LATE_AIRCRAFT_DELAY':0})
#bd4 = bd3.na.fill(0)
#bd4 = bd3.na.drop()

In [30]:
bd4.describe().toPandas().transpose()[0]

summary                count
YEAR                   30466
MONTH                  30466
DAY_OF_MONTH           30466
DAY_OF_WEEK            30466
CRS_DEP_TIME           30466
ARR_DELAY              30466
DEP_DELAY              30466
DISTANCE               30466
CANCELLED              30466
DIVERTED               30466
CARRIER_DELAY          30466
WEATHER_DELAY          30466
NAS_DELAY              30466
SECURITY_DELAY         30466
LATE_AIRCRAFT_DELAY    30466
Name: 0, dtype: object